<a href="https://colab.research.google.com/github/AnnaVitali/CP_programming_OR_tools/blob/master/CP_programming_OR_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CP programming with OR-Tools

COnstraint optimization, or constraint programming (CP), identifies feasible solutions out of a very large set of candidates, where the problem can be modeled in terms of arbitrary constraints. CP is based on feasibility (finding a feasible solution), rather rhan optimization (finding an optimal solution that maximize or minimize a certain objective function). However, CP can be used to solve optimization problems, simply by comparing the values of the objective function for all feasible solutions.

CP porblem may not even have an objective function, the goal may simply be to narrow down a very large set of possibile solutions to  more manageable subset by adding constrints to the problem.

In [1]:
%pip install ortools

In [28]:
from ortools.sat.python import cp_model
import random

The basic steps for setting up and solving a problem are the same:

1. Import the required libraries,
2. Declare the solver,
3. Create the variables,
4. Define the constraints,
5. Define the objective function,
6. Invoke the solver,
7. Display the result.

## Simple example
Let's start with a simple example problem in  which there are:
 - three variables: x, y and z each of which cam take on the values: 0, 1 or 2
 - one constraint x != y

In [22]:
# step 2 - deifning the model
model = cp_model.CpModel()

In [24]:
# step 3 - defining the variables
num_vals = 3

x = model.NewIntVar(0, num_vals - 1, "x")
y = model.NewIntVar(0, num_vals - 1, "y")
z = model.NewIntVar(0, num_vals - 1, "z")

In [25]:
# step 4 - defining the constraints
model.Add(x != y)

In [30]:
# step 5 - call the solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print(f"x = {solver.Value(x)}")
  print(f"y = {solver.Value(y)}")
  print(f"z = {solver.Value(z)}")
else:
  print("No solution found")

x = 1
y = 0
z = 0


### Finding all solution
In a problem like this you probably want to print all the feasible solutions or only a certain amount or part of them.

For doing this the main addition to the program above is a **solution printer** with a callback that you pass to the solver, which display each solution as it is found.

In [38]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
  """Print intermediate solutions."""

  def __init__(self, variables):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self.__variables = variables
    self.__solution_count = 0

  def on_solution_callback(self):
    self.__solution_count += 1
    print(f"Solution number {self.__solution_count}")
    for v in self.__variables:
      print(f"{v}={self.Value(v)}", end = " ")
      print(" ")
    print(" ")

  def solution_count(self):
    return self.__solution_count

class VarArraySolutionPrinterWithLimit(cp_model.CpSolverSolutionCallback):
  """Print intermediate solutions."""

  def __init__(self, variables, limit):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self.__variables = variables
    self.__solution_count = 0
    self.__solution_limit = limit

  def on_solution_callback(self):
    self.__solution_count += 1
    for v in self.__variables:
      print(f"{v} = {self.Value(v)}", end=" ")
      print()
      if self.__solution_count >= self.__solution_limit:
        print(f"Stop search after {self.__solution_limit} solutions")
        self.StopSearch()

  def solution_count(self):
    return self.__solution_count

In [36]:
solution_printer = VarArraySolutionPrinter([x, y, z])
# enumerate all solutions
solver.parameters.enumerate_all_solutions = True

# Solve
status = solver.Solve(model, solution_printer)

Solution number 1
x=1  
y=0  
z=0  
 
Solution number 2
x=2  
y=0  
z=0  
 
Solution number 3
x=2  
y=0  
z=1  
 
Solution number 4
x=1  
y=0  
z=1  
 
Solution number 5
x=2  
y=1  
z=1  
 
Solution number 6
x=2  
y=1  
z=0  
 
Solution number 7
x=2  
y=1  
z=2  
 
Solution number 8
x=2  
y=0  
z=2  
 
Solution number 9
x=1  
y=0  
z=2  
 
Solution number 10
x=0  
y=1  
z=2  
 
Solution number 11
x=0  
y=1  
z=1  
 
Solution number 12
x=0  
y=2  
z=1  
 
Solution number 13
x=0  
y=2  
z=2  
 
Solution number 14
x=1  
y=2  
z=2  
 
Solution number 15
x=1  
y=2  
z=1  
 
Solution number 16
x=1  
y=2  
z=0  
 
Solution number 17
x=0  
y=2  
z=0  
 
Solution number 18
x=0  
y=1  
z=0  
 


The previous section showed how to find all solutions to a CP problem Next, we'll show show how to find an optimal solution. As an example, we'll solve the following optimization problem.

Maximize `2x + 3y + 3z` subject to the following constraints

`x + 7/2y + 3/2z <= 25`

`3x - 5y + 7z <= 45`

`x, y, z >= 0`

In order to increase computational speed, the CP-SAT solver works over the integers. This means all constraints and the objective must have integer coefficient. The first constraint doesn't meet this condition. To solve the porblem, we must first transform the constraint by multiplying it by a sufficiently large integer to convert all the coefficients to integers.

We multiply the entire constraint by a sufficiently large integer to convert the coefficients to integers, in this case we multiply by 2

`2x + 7y + 3z <= 50`

In [37]:
model = cp_model.CpModel()

var_upper_bound = max(50, 45, 37)

# variables
x = model.NewIntVar(0, var_upper_bound, "x")
y = model.NewIntVar(0, var_upper_bound, "y")
z = model.NewIntVar(0, var_upper_bound, "z")

# constraint
model.Add(2 * x + 7 * y + 3 * z <= 50)
model.Add(3 * x - 5 * y + 7 * z <= 45)
model.Add(5 * x + 2 * y - 6 * z <= 37)

# objective function
model.Maximize(2 * x + 2 * y + 3 * z)

solver = cp_model.CpSolver()
status = solver.Solve(model)

solution_printer = VarArraySolutionPrinter([x, y, z])
# enumerate all solutions
solver.parameters.enumerate_all_solutions = True

# Solve
status = solver.Solve(model, solution_printer)

Solution number 1
x=8  
y=3  
z=4  
 
Solution number 2
x=7  
y=3  
z=5  
 


## Length of the strip problem

There are N rectangles numbered from 0 to N-1. The K-th rectangle has size A[K] × B[K].

We want to arrange as many rectangles as possible into a strip. The rectangles can be arranged into a strip if they all share a side of the same length (which becomes the height of the strip). Note that rectangles can be rotated.

Write a function:

class Solution { public int solution(int[] A, int[] B); }

that, given two arrays A and B of N integers each, returns the maximum number of rectangles that can be arranged into a strip.

In [3]:
A = [random.randint(1, 1000000) for i in range(1000)]
B = [random.randint(1, 1000000) for i in range(1000)]

model = cp_model.CpModel()

# we want to maximize the strip height
# we don't know what the hight'll gonna be, but this is a variable for our problem
strip_height = model.NewIntVar(1, 1000000000, "strip height")

in_strip_vars = []
rotated_in_strip_vars = []

for i in range(len(A)):
  # the value of the boolean variable will'be determined by the model
  in_strip_vars.append(model.NewBoolVar("rectangle %d in strip" % i))
  rotated_in_strip_vars.append(model.NewBoolVar("rectangle %d rotated" % i))

  # we need to define the constraint, if the rectangle is in the strip the hight of it will be the hhight
  # of the rectangle, if it is rotated it will be the width
  # these are constraints conditioned by bool variables
  model.Add(A[i] == strip_height).OnlyEnforceIf(in_strip_vars[i])
  model.Add(B[i] == strip_height).OnlyEnforceIf(rotated_in_strip_vars[i])

model.Maximize(sum(in_strip_vars + rotated_in_strip_vars)) # count the number of true bools

In [4]:
solver = cp_model.CpSolver()
solver.parameters.log_search_progress = True
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  for i in range(len(A)):
    if solver.Value(in_strip_vars[i]):
      print("in strip rectangle %d, dimension: %d x %d" % (i, B[i], A[i]))
    elif solver.Value(rotated_in_strip_vars[i]):
      print("in strip rectangle %d rotated, dimension: %d x %d" % (i, A[i], B[i]))

in strip rectangle 342 rotated, dimension: 182408 x 110968
in strip rectangle 854 rotated, dimension: 247708 x 110968


## Spread distance of the sheep's sunshade problem

There are N sheep relaxing in a field. They are positioned at points with integer coordinates. Each sheep has a square sunshade, so as not to overheat. The sunshade of a sheep standing at position (X, Y) spreads out to a distance of D from (X, Y), covering a square whose middle is at (X, Y) and whose sides are of length 2D. More precisely, it covers a square with vertices in points (X − D, Y − D), (X − D, Y + D), (X + D, Y − D) and (X + D, Y + D). Sheep are in the centres of their sunshades. Sunshades edges are parallel to the coordinate axes.

Every sheep spreads out its sunshade to the same width. No two sunshades can overlap, but their borders can touch.

What is the maximum integer width D to which the sheep can spread out their sunshades?

In [5]:
X = [1, 1 , 8]
Y = [1, 6, 0]

MAX_DISTANCE = 100000

model = cp_model.CpModel()

distance = model.NewIntVar(0, MAX_DISTANCE, "spread distance of the sunshade")
twice_distance = model.NewIntVar(0, 2 * MAX_DISTANCE, "two times the spread distance of the sunshade")
model.Add(twice_distance == 2 * distance)

#we need to define a costrain of non overlapping
width_intervals = []
height_intervals = []

for i in range(len(X)):
  left = model.NewIntVar(-MAX_DISTANCE, MAX_DISTANCE, "left")
  model.Add(left == X[i] - distance)
  right = model.NewIntVar(-MAX_DISTANCE, MAX_DISTANCE, "right")
  model.Add(right == X[i] + distance)

  width = model.NewIntervalVar(left, twice_distance, right, "width for shade %d" % i)
  width_intervals.append(width)

  top = model.NewIntVar(-MAX_DISTANCE, MAX_DISTANCE, "top")
  model.Add(top == Y[i] + distance)
  bottom = model.NewIntVar(-MAX_DISTANCE, MAX_DISTANCE, "bottom")
  model.Add(bottom == Y[i] - distance)

  height = model.NewIntervalVar(bottom, twice_distance, top, "hight for shade %d" % i)
  height_intervals.append(height)

model.AddNoOverlap2D(width_intervals, height_intervals)

# we need to define the goal
model.Maximize(distance)

In [6]:
solver = cp_model.CpSolver()
solver.parameters.log_search_progress = True
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  print("spread distance of the sunshade %d" % solver.Value(distance))
  for i in range(len(X) - 1):
    print("sheep in position (%d, %d)" % (X[i], Y[i]))

spread distance of the sunshade 2
sheep in position (1, 1)
sheep in position (1, 6)


## Building coverage problem

There are N rectangular buildings standing along the road next to each other. The K-th building is of size H[K] × 1.

Because a renovation of all of the buildings is planned, we want to cover them with rectangular banners until the renovations are finished. Of course, to cover a building, the banner has to be at least as high as the building. We can cover more than one building with a banner if it is wider than 1.

In [7]:
H = [5, 3, 2, 4]
NUM_BANNERS = 2

max_height = max(H)

model = cp_model.CpModel()

#we need to define a costrains we want to cover all the buildings and use as less material as possibile
#the part of the benner that don't cover any building should be the less possibile
#if we compute the area of the banners and we sum them toghether we want to minimize that sum
#we want to know where the first banner stops
banner_1_width = model.NewIntVar(1, len(H), "width first banner")
banner_2_width = model.NewIntVar(0, len(H), "width second banner")

banner_1_height = model.NewIntVar(1, max_height, "height first banner")
banner_2_height = model.NewIntVar(0, max_height, "height second banner")

model.Add(banner_2_width == len(H) - banner_1_width)


banner_1_area = model.NewIntVar(1, len(H) * max_height, "area first banner")
banner_2_area = model.NewIntVar(0, len(H) * max_height, "area second banner")

model.AddMultiplicationEquality(banner_1_area, [banner_1_width, banner_1_height])
model.AddMultiplicationEquality(banner_2_area, [banner_2_width, banner_2_height])

covered_by_banner1_vars = []
covered_by_banner2_vars = []

for x, h in enumerate(H):
  #is covered by banner 1 or 2
  covered_by_banner1 = model.NewBoolVar("building %d of height %d covered by banner 1" % (x, h))
  covered_by_banner2 = model.NewBoolVar("building %d of height %d covered by banner 2" % (x, h))

  covered_by_banner1_vars.append(covered_by_banner1)
  covered_by_banner2_vars.append(covered_by_banner2)

  #if banner 1 cover a building the same building can't be covered by number 2
  model.AddBoolOr([covered_by_banner1, covered_by_banner2])
  model.AddImplication(covered_by_banner1, covered_by_banner2.Not())
  model.AddImplication(covered_by_banner2, covered_by_banner1.Not())

  #if covered by banner 1
  model.Add(x < banner_1_width).OnlyEnforceIf(covered_by_banner1)
  model.Add(h <= banner_1_height).OnlyEnforceIf(covered_by_banner1)

  #if covered by banner 2
  model.Add(x >= banner_1_width).OnlyEnforceIf(covered_by_banner2)
  model.Add(h <= banner_2_height).OnlyEnforceIf(covered_by_banner2)

model.Minimize(banner_1_area + banner_2_area)

In [8]:
solver = cp_model.CpSolver()
solver.parameters.log_search_progress = True
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  print("area banner 1 %d" % solver.Value(banner_1_area))
  print("area banner 2 %d" % solver.Value(banner_2_area))

  for x, h in enumerate(H):
    if(solver.Value(covered_by_banner1_vars[x])):
      print("building %d covered by banner 1" % x)
    else:
      print("building %d civered by banner 2" % x)

area banner 1 5
area banner 2 12
building 0 covered by banner 1
building 1 civered by banner 2
building 2 civered by banner 2
building 3 civered by banner 2


## Multivitamin mix problem

Rick is really fond of fruit juices, but he is bored of their traditional flavours. Therefore, he has decided to mix as many of them as possible to obtain something entirely new as a result.

He has N glasses, numbered from 0 to N-1, each containing a different kind of juice. The J-th glass has capacity[J] units of capacity and contains juice[J] units of juice. In each glass there is at least one unit of juice.

Rick want to create a multivitamin mix in one of the glasses. He is going to do it by pouring juice from several other glasses into the chosen one. Each of the used glasses must be empty at the end (all of the juice from each glass has to be poured out).

What is the maximum number of flavours that Rick can mix?

In [11]:
juice = [10, 2, 1, 1]
capacity = [10, 3, 2, 2]

MAX_NUMBER = 1000000


model = cp_model.CpModel()
# galss[0, 1, 2, 3]
# the capacity are fix we don't need to represent that we need to represent the sum of something
# everything that goes on the first glass needs to add up o something that less than 10

# for each juisce we want to know where it will go
destination_glass_matrix = []
# that because th solution want us to tell which juice moved to a different glass
moved_to_self = []
for i in range(len(juice)):
  destination_glass_row = []
  moved_to_self.append(model.NewBoolVar("moved to self %d" % i))
  for j in range(len(juice)):
    destination_glass_row.append(model.NewBoolVar("juice %d is moving to glass %d" % (i, j)))
    # the juice can move only to one glass the row sum of the matrix should be 1
  model.Add(sum(destination_glass_row) == 1)
  model.Add(moved_to_self[i] == destination_glass_row[i])
  destination_glass_matrix.append(destination_glass_row)

#if we take a row of the matrix all the true in that row means tht juice is moving to that glass
for i in range(len(juice)):
  # i represent a particular glass with a capacity
  # each true in that row means that juice moves here
  # so we sum the juice that moved here and make sure it's less than capacity
  capacity_moved = model.NewIntVar(0, MAX_NUMBER, "filled capacity for this glass %d" % i)
  # all the juice need to be in glasses
  model.Add(capacity_moved == cp_model.LinearExpr.WeightedSum(destination_glass_matrix[i], juice))
  model.Add(capacity_moved <= capacity[i])


model.Maximize(sum([boolvar for row in destination_glass_matrix for boolvar in row]) - sum(moved_to_self))

In [13]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  for i in range(len(juice)):
    for j in range(len(juice)):
      if solver.Value(destination_glass_matrix[i][j]):
        print("juice %d moved to glass %d" % (i, j))

juice 0 moved to glass 1
juice 1 moved to glass 3
juice 2 moved to glass 3
juice 3 moved to glass 2
